# Tree Reconstruction Notebook

This notebook provides code to reconstruct the skeleton from a tree point cloud.

---------------

##### Imports

In [3]:
# Add project src to path.
import set_path

# Import modules.
import os
import numpy as np
import open3d as o3d
import logging as log
import utils.tree_utils as tree_utils
import utils.o3d_utils as o3d_utils

from labels import Labels

### 1. Load Data
---

Load tree point cloud

In [4]:
tree_cloud = o3d_utils.read_las('../../dataset/cyclo/filtered_tree_121913_487434.las')
labels = np.full(len(tree_cloud.points), Labels.WOOD)

In [5]:
o3d.visualization.draw_geometries([tree_cloud])

Load executable

In [6]:
adTree_exe = '../../AdTree/build/bin/AdTree.app/Contents/MacOS/AdTree'

### 2. Reconstruction
---
![Cyclo Media Reconstruction](../../imgs/cyclo_reconstruction.png)

**Leaf Wood Classification** (optional)

The _method_ parameter in `leafwood_classificiation()` can take the following values:
- _'curvature'_
- _'survace_variation'_ (default)

In [ ]:
# Leaf woord classification using 'curvature'
labels = tree_utils.leafwood_classificiation(tree_cloud, method='curvature')
print(f"Labeled {np.sum(labels==Labels.WOOD)}/{len(labels)} points as wood.")
tree_utils.show_tree(tree_cloud, labels)

In [ ]:
# Leaf woord classification using 'surface_variation'
labels = tree_utils.leafwood_classificiation(tree_cloud, method='surface_variation')
print(f"Labeled {np.sum(labels==Labels.WOOD)}/{len(labels)} points as wood.")
tree_utils.show_tree(tree_cloud, labels)

**Skeleton Reconstruction**

In [7]:
# filter labelled wood from leaves (see previous step)
wood_cloud = tree_cloud.select_by_index(np.where(labels==Labels.WOOD)[0])

# WARNING: reconstruction might take some minutes.
skeleton = tree_utils.reconstruct_skeleton(wood_cloud, adTree_exe)

In [8]:
tree_utils.show_tree(tree_cloud, labels, skeleton)